In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

%matplotlib inline
plt.style.use('seaborn-ticks')
plt.rcParams.update({'font.size': 14})

X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],  # стаж репетитора
              [500, 700, 750, 600, 1450,        # средняя стоимость занятия
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]],  # квалификация репетитора
              dtype = np.float64).T 

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1]) # поступил или нет ученик на специальность Математика

def standardization(x: np.ndarray) -> np.ndarray:
    s = (x - x.mean()) / x.std()
    return s

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def calc_logloss(y, y_pred):
    err = np.mean(- y * np.log(y_pred) - (1.0 - y) * np.log(1.0 - y_pred))
    return err

def eval_LR_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    w = np.random.randn(X.shape[1])
    n = X.shape[0]
    for i in range(1, iterations + 1):
        z = np.dot(X, w) # log(p/(1-p))
        y_pred = sigmoid(z) # p [0, 1]
        err = calc_logloss(y, y_pred)
        w -= alpha * (1/n * np.dot((y_pred - y), X))
        if i % (iterations / 10) == 0:
            print(i, w, err)
    return w

### Задача 1

Измените функцию `calc_logloss` так, чтобы нули по возможности не попадали в `np.log` (как вариант - использовать `np.clip` или `np.where`). 

#### Решение

In [2]:
def calc_logloss(y, y_pred, y_min=1e-8):
    y_pred_opt = np.clip(y_pred, a_min=y_min, a_max=1-y_min)
    err = np.mean(- y * np.log(y_pred_opt) - (1.0 - y) * np.log(1.0 - y_pred_opt))
    return err

### Задача 2

На данных из урока изучите влияние гиперпараметров на ошибку алгоритма. Подберите аргументы функции `eval_LR_model` для логистической регрессии таким образом, чтобы log loss не превышал значение `0.3` `(0.1).` Как изменились веса?

#### Решение

In [3]:
def eval_LR_model_new(X, y, iterations, alpha=1e-4, diff=0.1):
    np.random.seed(42)
    w = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(1, iterations + 1):
        z = np.dot(X, w) # log(p/(1-p))
        y_pred = sigmoid(z) # p [0, 1]
        err = calc_logloss(y, y_pred)
        w -= alpha * (1/n * np.dot((y_pred - y), X))
        if err < diff:
            break
        if i % (iterations / 10) == 0:
            print(i, w, err)

    return w

X_st = X.copy()
X_st[:, 2] = standardization(X[:, 2])

#Выполняем пдбор параметров
for iterations in range(1000, 10000, 1000):
  for alpha in [0.0001, 0.001, 0.01, 0.1, 1, 2]:
      print(iterations, alpha)      
      w = eval_LR_model_new(X_st, y, iterations, alpha)



1000 0.0001
100 [ 0.49282748 -0.15007528  0.64748973  1.51727915] 1.2014814214705334
200 [ 0.48896219 -0.16184918  0.64728128  1.51155738] 1.1828456288538924
300 [ 0.48511874 -0.17358386  0.64706349  1.50586552] 1.1643525542846556
400 [ 0.4812976  -0.18527698  0.64683669  1.50020462] 1.1460086359433084
500 [ 0.47749927 -0.19692597  0.64660127  1.4945758 ] 1.127820879406358
600 [ 0.47372426 -0.20852799  0.6463577   1.48898028] 1.109796908143704
700 [ 0.46997312 -0.22007992  0.6461065   1.48341934] 1.0919450148769096
800 [ 0.46624642 -0.23157833  0.64584825  1.47789438] 1.074274212586137
900 [ 0.46254476 -0.24301946  0.64558365  1.4724069 ] 1.0567942835649755
1000 [ 0.45886878 -0.25439917  0.64531344  1.46695851] 1.0395158244739489
1000 0.001
100 [ 0.4588585  -0.2544203   0.6453169   1.46694374] 1.0410271098401283
200 [ 0.42366472 -0.36366056  0.64254667  1.41503365] 0.8819616824356442
300 [ 0.39197407 -0.45912506  0.6408019   1.36968388] 0.7595098840306068
400 [ 0.36451116 -0.53324936  

Таким образом в нашем случае  точность 0.1 достигается при alpha = 2 и количестве итераций 7000 

Таким образом в нашем случае  точность 0.3 достигается при alpha = 1 и количестве итераций 400

При этом часть весов теряет силу, один вес - становится слишком значимым. Я бы назвал такую модель переобученной

### Задача 3

Создайте функцию `calc_pred_proba`, возвращающую предсказанную вероятность класса "1". На вход функции подаются значения признаков Х и веса, которые уже посчитаны функцией `eval_LR_model`.

#### Решение

In [4]:
w = eval_LR_model_new(X_st, y, 1000, alpha=1e-2, diff=0.1)

def calc_pred_proba(W, X):
    return sigmoid(W @ X.T)
calc_pred_proba(w, X_st)


100 [ 0.25769171 -0.68297243  0.68841418  1.2405903 ] 0.5908596023453088
200 [ 0.14805533 -0.69317972  0.77965488  1.21287192] 0.5687278067099054
300 [ 0.06257746 -0.68605867  0.85181735  1.21283095] 0.5560473747533587
400 [-0.00887935 -0.68415387  0.90713736  1.22443854] 0.5476800278385013
500 [-0.0701621  -0.68661758  0.94984062  1.24404686] 0.5416765996151559
600 [-0.1239972  -0.69202069  0.98318257  1.26924759] 0.5369841864977791
700 [-0.17239277 -0.69933957  1.00953865  1.29824779] 0.5330427663756572
800 [-0.21681092 -0.7078635   1.03062761  1.32973537] 0.5295551503580799
900 [-0.25830834 -0.71709934  1.04769231  1.36276535] 0.5263621224044184
1000 [-0.29764618 -0.72670545  1.061634    1.39666497] 0.5233765331724747


array([0.33902271, 0.44280838, 0.63405613, 0.38966699, 0.79503275,
       0.67169769, 0.80349092, 0.24405607, 0.31494039, 0.74969905])

### Задача 4

Создайте функцию `calc_pred`, возвращающую предсказанные классы (0 или 1). На вход функции подаются значения признаков Х и веса, которые уже посчитаны функцией `eval_LR_model`, а также порог вероятности.

#### Решение

In [5]:
def calc_pred(W, X, bound=0.5):
    y_pred = sigmoid(W @ X.T)
    y_pred = np.where(y_pred > bound, 1, 0)

    return y_pred
y_pred = calc_pred(w, X_st)

### Задача 5

(\*) Напишите функции для расчета accuracy, матрицы ошибок, precision и recall, а также F1-score.

#### Решение

In [6]:
def accurency_score(y_true, y_pred):
    return (y_true == y_pred).sum() / y_true.shape[0]

In [7]:
print(accurency_score(y, y_pred))

0.8


In [8]:
def err_matrix(y_true, y_pred):
    TP = y_true[(y_true - y_pred) == 0].sum().astype(int)
    FP = ((y_true - y_pred) == -1).sum().astype(int)
    TN = (y_true[(y_true - y_pred) == 0]==0).sum().astype(int)
    FN = ((y_true - y_pred) == 1).sum().astype(int)
    return TP, FP, TN, FN
print(err_matrix(y, y_pred))

(4, 1, 4, 1)


In [9]:
def precision_score(y_true, y_pred):
    TP, FP, _, _ = err_matrix(y_true, y_pred)
    return TP / (TP + FP)
precision_score(y, y_pred)

0.8

In [10]:
def recall_score(y_true, y_pred):
    TP, _, _, FN = err_matrix(y_true, y_pred)
    return TP / (TP + FN)
recall_score(y, y_pred)

0.8

In [11]:
def f1_score(y_true, y_pred):
    return round(2 * precision_score(y_true, y_pred) * recall_score(y_true, y_pred) / (precision_score(y_true, y_pred) + recall_score(y_true, y_pred)), 4)
f1_score(y, y_pred)

0.8